In [1]:
cd ..

/Users/m.dee/Documents/ML-Navigator


<img src="flow_3.png">

In [2]:
from flows.flows import Flows

Welcome to the Data Science Package. First create an object as follows:
For example, use the code below to import the flow 0:
flow = Flows(0)


In [3]:
flow = Flows(3)

Please use the following function to read the data
dataframe_dict = flow.load_data(path: str, files_list: list)
For example: path = './data'
For example: files_list = ['train.csv','test.csv']
The output is a dictionary that contains dataframes e.g.  
dataframe_dict = {'train': train_dataframe,'test': test_dataframe}


In [4]:
path = "./data/flow_3"
files_list = ['train_transaction.csv','test_transaction.csv']

In [5]:
dataframe_dict, columns_set = flow.load_data(path, files_list, rows_amount=1500)

The summary of the train_transaction dataset:
The train_transaction dataset contains 394 columns
There are 14 string categorical columns
There are 326 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
There are 54 columns with numeric continuous values 
There are 0 columns that contain date
There are 0 columns that contain valid nested JSON data
There are 0 columns that contain other type of data
********** End of the summary of the train_transaction dataset **********
The summary of the test_transaction dataset:
The test_transaction dataset contains 393 columns
There are 14 string categorical columns
There are 325 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: 

In [6]:
dataframe_dict["train_transaction"].head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
 columns_set["train_transaction"].keys()

dict_keys(['categorical_string', 'categorical_integer', 'continuous', 'date', 'json', 'other'])

In [9]:
dataframe_dict, columns_set = flow.encode_categorical_feature(dataframe_dict,print_results=10)

The reference dataframe is: train_transaction
the are 2 datasets provided
encoding the feature in the dataset test_transaction
the number of classes in ProductCD feature is: 5
the are 2 datasets provided
encoding the feature in the dataset test_transaction
the number of classes in card3 feature is: 19
the are 2 datasets provided
encoding the feature in the dataset test_transaction
the number of classes in card4 feature is: 4
the are 2 datasets provided
encoding the feature in the dataset test_transaction
the number of classes in card5 feature is: 32
the are 2 datasets provided
encoding the feature in the dataset test_transaction
the number of classes in card6 feature is: 2
the are 2 datasets provided
encoding the feature in the dataset test_transaction
the number of classes in addr2 feature is: 3
the are 2 datasets provided
encoding the feature in the dataset test_transaction
the number of classes in dist2 feature is: 74
the are 2 datasets provided
encoding the feature in the dataset t

In [ ]:
ignore_columns = ['isFraud']

In [ ]:
dataframe_dict, columns_set = flow.drop_columns_constant_values(dataframe_dict, ignore_columns)

In [ ]:
dataframe_dict, columns_set = flow.drop_correlated_columns(dataframe_dict, ignore_columns)

In [ ]:
ignore_columns = ["TransactionID", "isFraud"]

In [ ]:
dataframe_dict, columns_set = flow.scale_data(dataframe_dict, ignore_columns)

In [ ]:
flow.exploring_data(dataframe_dict, "test_transaction")

In [ ]:
flow.comparing_statistics(dataframe_dict)

In [ ]:
columns = dataframe_dict["train_transaction"].columns
train_dataframe = dataframe_dict["train_transaction"][
    [x for x in columns_set["train_transaction"]["continuous"] if x not in ignore_columns]]
test_dataframe = dataframe_dict["test_transaction"][
    [x for x in columns_set["train_transaction"]["continuous"] if x not in ignore_columns]]
train_target = dataframe_dict["train_transaction"]["isFraud"]

In [ ]:
parameters = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # foldnr:5 , "split_ratios": 0.8 # "split_ratios":(0.7,0.2)
    },
    "model": {"type": "Logistic regression",
              "hyperparameters": {"alpha": "optimize",  # alpha:optimize
                                  },
              },
    "metrics": ["r2_score", "mean_squared_error"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [ ]:
model_index_list, save_models_dir, y_test = flow.training(parameters)

In [ ]:
parameters_lighgbm = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # foldnr:5 , "split_ratios": 0.8 # "split_ratios":(0.7,0.2)
    },
    "model": {"type": "lightgbm",
              "hyperparameters": dict(objective='binary', metric='cross-entropy', num_leaves=5,
                                      boost_from_average=True,
                                      learning_rate=0.05, bagging_fraction=0.99, feature_fraction=0.99, max_depth=-1,
                                      num_rounds=10000, min_data_in_leaf=10, boosting='dart')
              },
    "metrics": ["r2_score", "mean_squared_error"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [ ]:
model_index_list, save_models_dir, y_test = flow.training(parameters_lighgbm)